# Tutorial 4: Linear Operators, Duals, and Adjoints

In previous tutorials, we introduced the `HilbertSpace` and the concept of a `dual` space. Now, we'll focus on the `LinearOperator` class, which defines linear maps between these spaces.

### The Theory in Outline

Given a linear operator $A: X \to Y$, we can define two related operators:

1.  **The Dual Operator ($A'$)**: The dual (or transpose) operator maps between the dual spaces, $A': Y' \to X'$. It is defined by the relation:
    $$
    \langle y', A(x) \rangle = \langle A'(y'), x \rangle
    $$
    This relation holds for any $x \in X$ and $y' \in Y'$.

2.  **The Adjoint Operator ($A^*$)**: The adjoint operator maps between the original primal spaces, but in reverse, $A^*: Y \to X$. It is defined by the inner product:
    $$
    \langle y, A(x) \rangle_Y = \langle A^*(y), x \rangle_X
    $$

A key feature of this library is that if you provide just the forward mapping for `A`, the library can automatically derive its `dual` and `adjoint` operators.

In this tutorial, we will:
1.  Create a `LinearOperator` from a simple mapping.
2.  Combine operators using standard arithmetic operations.
3.  Numerically verify the defining relations for the dual and adjoint.
4.  Create an operator on a `Sobolev` space using `from_formal_adjoint`.

In [ ]:
# To run in colab, uncomment the line below to install pygeoinf. 
#%pip install pygeoinf

import numpy as np
import pygeoinf as inf
import matplotlib.pyplot as plt
from pygeoinf.symmetric_space.circle import Sobolev
from pygeoinf import EuclideanSpace

# For reproducibility
np.random.seed(123)

## 1. Defining a `LinearOperator`

The most direct way to create a `LinearOperator` is from its constituent parts: a `domain` space, a `codomain` space, and a `mapping` function that defines its action.

Let's create a simple differentiation operator on a space of functions on the circle. In the Fourier basis, differentiation is just multiplication by $ik$.

In [ ]:
# Create a Sobolev space of smooth functions on the circle
model_space = Sobolev.from_sobolev_parameters(2.0, 0.1)

# Define the mapping for a differentiation operator
def diff_map(u):
    # Go to Fourier domain (coefficients)
    coeff = model_space.to_coefficient(u)
    k = np.arange(coeff.size)
    # Multiply by ik (where i is sqrt(-1))
    diff_coeff = 1j * k * coeff
    # Transform back to the spatial domain
    return model_space.from_coefficient(diff_coeff)

# Create the LinearOperator
diff_op = inf.LinearOperator(model_space, model_space, diff_map)

# Let's test it on f(theta) = sin(2*theta)
# The derivative should be 2*cos(2*theta)
test_func = model_space.project_function(lambda theta: np.sin(2*theta))
derivative_func = diff_op(test_func)

# Plot the results to verify
fig, ax = model_space.plot(test_func, label='sin(2θ)')
model_space.plot(derivative_func, fig=fig, ax=ax, label='d/dθ [sin(2θ)]')
ax.plot(model_space.angles(), 2 * np.cos(2 * model_space.angles()), 'k--', label='Expected: 2cos(2θ)')
ax.legend()
ax.set_title("Differentiation Operator Test")
plt.show()

## 2. Operator Arithmetic

`LinearOperator` objects support standard arithmetic operations, allowing you to build complex operators from simpler ones. You can use:
* `+` and `-` for addition and subtraction.
* `*` for scalar multiplication.
* `@` for composition (matrix multiplication).

In [ ]:
# Define two simple operators on EuclideanSpace
space = EuclideanSpace(2)
M1 = np.array([[1, 2], [3, 4]])
M2 = np.array([[0, 1], [1, 0]])

A = inf.LinearOperator.from_matrix(space, space, M1)
B = inf.LinearOperator.from_matrix(space, space, M2)

# --- Demonstrate arithmetic ---

# Addition: C = A + B
C = A + B
print(f"Matrix of C = A + B:\n{C.matrix(dense=True)}\n")

# Scalar Multiplication: D = 3 * A
D = 3 * A
print(f"Matrix of D = 3 * A:\n{D.matrix(dense=True)}\n")

# Composition: E = A @ B
E = A @ B
print(f"Matrix of E = A @ B:\n{E.matrix(dense=True)}\n")

# We can verify the composition result
assert np.allclose(E.matrix(dense=True), M1 @ M2)

## 3. Duals and Adjoints

The concepts of dual and adjoint operators are easiest to understand in a simple `EuclideanSpace`.

### Providing Mappings vs. Automatic Derivation

When creating a `LinearOperator`, you can optionally provide the `dual_mapping` and `adjoint_mapping` directly in the constructor. This is highly recommended if you know them, as it is much more efficient.

If you **do not** provide them, the library will derive them for you. The default derivation, however, can be slow as it may involve evaluating the operator on every basis vector of the space to construct an internal matrix representation. For high-dimensional spaces, this is very inefficient.

Let's create an operator and verify the defining relations.

In [ ]:
# Define domain and codomain
domain = EuclideanSpace(3)
codomain = EuclideanSpace(2)

# Create an operator from a matrix
M = np.array([[1, 2, 3], [4, 5, 6]])
# We know the adjoint is the transpose, so we can provide it for efficiency
adjoint_map = lambda y: M.T @ y
A = inf.LinearOperator(domain, codomain, lambda x: M @ x, adjoint_mapping=adjoint_map)

# Get the dual and adjoint operators
A_dual = A.dual
A_adjoint = A.adjoint

# Create random vectors for testing
x = domain.random()
y = codomain.random()

# --- Verification ---

# 1. Dual Relation: <y', A(x)> = <A'(y'), x>
# For Euclidean space, the dual of y is a LinearForm with the same components
y_prime = codomain.to_dual(y) 
lhs_dual = codomain.duality_product(y_prime, A(x))
rhs_dual = domain.duality_product(A_dual(y_prime), x)

print(f"--- Dual Operator Verification ---")
print(f"<y', A(x)> = {lhs_dual:.4f}")
print(f"<A'(y'), x> = {rhs_dual:.4f}")
assert np.isclose(lhs_dual, rhs_dual)

# 2. Adjoint Relation: <y, A(x)>_Y = <A*(y), x>_X
lhs_adjoint = codomain.inner_product(y, A(x))
rhs_adjoint = domain.inner_product(A_adjoint(y), x)

print(f"\n--- Adjoint Operator Verification ---")
print(f"<y, A(x)>_Y = {lhs_adjoint:.4f}")
print(f"<A*(y), x>_X = {rhs_adjoint:.4f}")
assert np.isclose(lhs_adjoint, rhs_adjoint)

## 4. Adjoints in Function Spaces (`from_formal_adjoint`)

In a simple `EuclideanSpace`, the dual and adjoint are effectively the same. This is not true for more complex spaces, like a `Sobolev` space, which has a non-trivial inner product.

A common task is to define an operator in terms of its simple action on the underlying L² space of functions, and then "lift" it to be a proper operator on the `Sobolev` space with the correct adjoint. The `from_formal_adjoint` static method is designed for exactly this purpose.

In [ ]:
# Our Sobolev space from before
sobolev_space = model_space

# 1. Define the differentiation operator on the underlying Lebesgue (L2) space
# This is simpler as we don't have to worry about the Sobolev weighting yet.
lebesgue_space = sobolev_space.underlying_space
# In L2 space, the adjoint of d/dθ is -d/dθ. We can set it manually.
diff_op_L2 = inf.LinearOperator(lebesgue_space, lebesgue_space, diff_map,
                                adjoint_mapping = lambda u: -1 * diff_map(u))

# 2. Use from_formal_adjoint to "lift" this operator to the Sobolev space
# This creates a new operator with the correct adjoint for the Sobolev inner product.
diff_op_sobolev = inf.LinearOperator.from_formal_adjoint(sobolev_space, sobolev_space, diff_op_L2)

# 3. Verify the adjoint property in the Sobolev space
f = sobolev_space.random()
g = sobolev_space.random()

lhs = sobolev_space.inner_product(g, diff_op_sobolev(f))
rhs = sobolev_space.inner_product(diff_op_sobolev.adjoint(g), f)

print(f"--- Sobolev Adjoint Verification ---")
print(f"<g, A(f)>_Sobolev = {lhs:.4f}")
print(f"<A*(g), f>_Sobolev = {rhs:.4f}")
# The results should be close, allowing for numerical precision of the FFTs.
assert np.isclose(lhs, rhs, rtol=1e-5)